In [82]:
import sqlite3 as db
import pandas as pd
import featuretools as ft
import json

In [83]:
# Hacer una query SQL
def sql_query(q):
    conn = db.connect('../db/sqlite/eicu_v2_0_1.sqlite3/eicu_v2_0_1.sqlite3')
    df = pd.read_sql_query(q, conn)
    conn.close()
    
    return df

# Leer todos los CSV
def read_csvs():
    #import os
    #print( os.getcwd())
    datasets = [ 'admissiondrug', 'admissionDx', 'allergy', 'apacheApsVar', 'apachePatientResult', 'apachePredVar', 'carePlanCareProvider', 'carePlanEOL', 'carePlanGeneral',
                 'carePlanGoal', 'carePlanInfectiousDisease', 'customLab', 'diagnosis', 'hospital', 'infusiondrug', 'intakeOutput', 'lab', 'medication', 'microLab', 'note',
                 'nurseAssessment', 'nurseCare', 'nurseCharting', 'pastHistory', 'patient', 'physicalExam', 'respiratoryCare', 'respiratoryCharting', 'treatment', 'vitalAperiodic',
                 'vitalPeriodic']

    dfs = {}

    for ds_name in datasets:
        dfs[ds_name] = pd.read_csv('../db/csv/' + ds_name + '.csv')
    
    return dfs

dfs = read_csvs()
has_dropped_keys = False # Para que no se droppeen cada vez que se ejecuta la celda siguiente

C:\Users\aleja\AppData\Local\Temp\ipykernel_1444\759616273.py:21: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs[ds_name] = pd.read_csv('../db/csv/' + ds_name + '.csv')


In [84]:
# Dropear IDs de todas las tablas (primera columna)
def drop_keys(dfs):
    for df_key in dfs.keys():
        if df_key not in ('hospital', 'patient'): # No dropear hospitalId o patientUnitStayId
            df = dfs[df_key]
            dfs[df_key] = df.drop(columns=[df.columns.values[0]])

if not has_dropped_keys:
    drop_keys(dfs)
    has_dropped_keys = True

In [85]:
dfs['respiratoryCare']

,patientunitstayid,respcarestatusoffset,currenthistoryseqnum,airwaytype,airwaysize,airwayposition,cuffpressure,ventstartoffset,ventendoffset,priorventstartoffset,...,peeplimit,cpaplimit,setapneainterval,setapneatv,setapneaippeephigh,setapnearr,setapneapeakflow,setapneainsptime,setapneaie,setapneafio2
0,147784,1188,2,NaN,NaN,NaN,NaN,0,0,-361,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,147784,-61,1,NaN,NaN,NaN,NaN,-361,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,165840,-63,1,NaN,NaN,NaN,NaN,-363,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,165840,73,2,NaN,NaN,NaN,NaN,0,0,-363,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,187150,7293,2,NaN,NaN,NaN,NaN,0,0,-522,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5431,3353113,3777,7,No Artificial Airway,NaN,NaN,NaN,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5432,3353113,204,2,No Artificial Airway,NaN,NaN,NaN,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5433,3353113,369,3,No Artificial Airway,NaN,NaN,NaN,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5434,3353113,-1311,1,No Artificial Airway,NaN,NaN,NaN,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
def clean_respiratoryCare(row, df):
    respiratoryCare = row['currenthistoryseqnum']
        
   
    return  df[df['patientunitstayid']==row['patientunitstayid']]['currenthistoryseqnum'].max()
  

In [87]:
# TODO: Mirar relación entre labmeasurenamesystem y labmeasurenameinterface DONE

dfs['respiratoryCare']['currenthistoryseqnum']     = dfs['respiratoryCare'].apply(lambda row : clean_respiratoryCare(row, dfs['respiratoryCare']), axis=1)

dfs['respiratoryCare'] = dfs['respiratoryCare'].drop(columns=[
    'airwaysize', 'airwayposition', 'cuffpressure', 'apneaparams', 'lowexhmvlimit', 'hiexhmvlimit', 'lowexhtvlimit', 'hipeakpreslimit', 'lowpeakpreslimit',
    'hirespratelimit', 'lowrespratelimit', 'sighpreslimit', 'lowironoxlimit', 'highironoxlimit', 'meanairwaypreslimit', 'peeplimit', 'cpaplimit',
    'setapneainterval', 'setapneatv', 'setapneaippeephigh', 'setapnearr', 'setapneapeakflow', 'setapneainsptime', 'setapneaie', 'setapneafio2', 
    'respcarestatusoffset', 'airwaytype', 'ventstartoffset', 'ventstartoffset', 'ventstartoffset', 'priorventendoffset', 'ventendoffset', 'priorventstartoffset'
], axis=1)

dfs['respiratoryCare'] = dfs['respiratoryCare'].drop_duplicates()

In [88]:
dfs['respiratoryCare']

,patientunitstayid,currenthistoryseqnum
0,147784,2
2,165840,2
4,187150,2
6,197617,2
7,197619,1
...,...,...
5021,3351763,212
5233,3352230,189
5418,3352231,4
5422,3352333,7


In [89]:
dfs['respiratoryCare'].info()
print()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 395 entries, 0 to 5429
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   patientunitstayid     395 non-null    int64
 1   currenthistoryseqnum  395 non-null    int64
dtypes: int64(2)
memory usage: 9.3 KB

